# Holiday Farm Fire Burn Severity

###### Kate Emmel

This project is based off of https://github.com/Esri/arcgis-python-api/blob/master/samples/04_gis_analysts_data_scientists/calculate_post_fire_landslide_risk.ipynb. I will be looking at a different fire with this code. This project will look at landsat imagery from before and after the Holiday Farm Fire of 2020. This fire began on September 7th, 2020 and lasted until October 29th of 2020. It severly affected multiple communities and is one of the largest burning fires in Oregon's recorded history. It burned about 170,000 acres. 

In [1]:
import arcgis
from arcgis.gis import GIS
from arcgis.raster.functions import *
from ipywidgets import *

import arcpy
from arcpy.ia import *
from arcpy import env
from arcpy.sa import *
#Usually, you do not want to import * but define the specific function you will use. 

import matplotlib.pyplot as plt


### Getting Data

I found data from before the fire, taken on August 23rd, 2020 from USGS Earth explorer data base. https://earthexplorer.usgs.gov/

I then found a landsat imagery taken October 26th 2020, just a few days before the fire was officially put out. I chose this data because it was relatively free of smoke and clouds.
These are both Landsat 8. 

I will also need a DEM (digital elevation model) and a NLCD (National Land cover Dataset which will show land use and vegetationn type. I have sourced these from ESRI's GIS content.

In [2]:
bl8band2 = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\Raster Data\LC08_L2SP_046029_20200823_20200905_02_T1_SR_B2.tif'
bl8band3 = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\Raster Data\LC08_L2SP_046029_20200823_20200905_02_T1_SR_B3.tif'
bl8band2raster = plt.imread(bl8band2)
bl8band3raster = plt.imread(bl8band3)

In [3]:
al8band2 = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\Raster Data\LC08_L2SP_046029_20201026_20201106_02_T1_SR_B2.tif'
al8band3 = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\Raster Data\LC08_L2SP_046029_20201026_20201106_02_T1_SR_B3.tif'
al8band2raster = plt.imread(al8band2)
al8band3raster = plt.imread(al8band3)

In [4]:
bl8band5 = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\Raster Data\LC08_L2SP_046029_20200823_20200905_02_T1_SR_B5.tif'
bl8band5raster = plt.imread(bl8band5)
bl8band7 = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\Raster Data\LC08_L2SP_046029_20200823_20200905_02_T1_SR_B7.tif'
bl8band7raster = plt.imread(bl8band7)
bl8band5raster

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [5]:
bl8band7raster

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [6]:
al8band5 = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\Raster Data\LC08_L2SP_046029_20201026_20201106_02_T1_SR_B5.tif'
al8band7 = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\Raster Data\LC08_L2SP_046029_20201026_20201106_02_T1_SR_B7.tif'
al8band5raster = plt.imread(al8band5)
al8band7raster = plt.imread(al8band7)
al8band5raster

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [7]:
al8band7raster

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

In [8]:
gis = GIS('https://pythonapi.playground.esri.com/portal', 'arcgis_python', 'amazing_arcgis_123')
arcgis.raster.analytics.is_supported(gis)

True

In [9]:
dem = gis.content.search('title:Sonoma_DEM owner:api_data_owner',
                         item_type = "Image Service",
                         outside_org=True)[0].layers[0]
nlcd = gis.content.search('title:Sonoma_NLCD2011 owner:api_data_owner',
                          item_type = "Image Service",
                          outside_org=True)[0].layers[0]
basins = gis.content.search('title:Sonoma_Basins owner:api_data_owner',
                            item_type = "Image Service",
                            outside_org=True)[0].layers[0]

## Create a burn severity map
To compare the burn scars on the before and after Landsat imagery, we’ll choose the multispectral bands 5,3,2 to be displayed. The [5,3,2] band combination improves visibility of fire and burn scars. Healthy vegetation is shown in bright red, while stressed vegetation is displayed as dull red. Nonvegetated features such as bare and urban areas are displayed in various shades of gray and blue.

Below, we apply the same bands combination to the before_l8 and after_l8 layers.


In [11]:
infrared_before = (bl8band5 + bl8band3 + bl8band2)
infrared_after = (al8band5 + al8band3 + al8band2)

In [12]:
infrared_before

'C:\\Users\\emmel\\Desktop\\Notebooks\\AWS Mapping\\HolidayFarmFire\\HHFire Data\\Raster Data\\LC08_L2SP_046029_20200823_20200905_02_T1_SR_B5.tifC:\\Users\\emmel\\Desktop\\Notebooks\\AWS Mapping\\HolidayFarmFire\\HHFire Data\\Raster Data\\LC08_L2SP_046029_20200823_20200905_02_T1_SR_B3.tifC:\\Users\\emmel\\Desktop\\Notebooks\\AWS Mapping\\HolidayFarmFire\\HHFire Data\\Raster Data\\LC08_L2SP_046029_20200823_20200905_02_T1_SR_B2.tif'

In [13]:
infrared_after

'C:\\Users\\emmel\\Desktop\\Notebooks\\AWS Mapping\\HolidayFarmFire\\HHFire Data\\Raster Data\\LC08_L2SP_046029_20201026_20201106_02_T1_SR_B5.tifC:\\Users\\emmel\\Desktop\\Notebooks\\AWS Mapping\\HolidayFarmFire\\HHFire Data\\Raster Data\\LC08_L2SP_046029_20201026_20201106_02_T1_SR_B3.tifC:\\Users\\emmel\\Desktop\\Notebooks\\AWS Mapping\\HolidayFarmFire\\HHFire Data\\Raster Data\\LC08_L2SP_046029_20201026_20201106_02_T1_SR_B2.tif'

In [14]:
#trying to look at full tif of all bands
#after_fire = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\PostFireTotal.tif'
#before_fire = r'C:\Users\emmel\Desktop\Notebooks\AWS Mapping\HolidayFarmFire\HHFire Data\PreFireTotal.TIF'
#bl8 = plt.imread(before_fire)
#bl8

## Quantitative Assessment
A Normalized Burn Ratio (NBR) can be used to delineate the burned areas and identify the severity of the fire. The formula for NBR is very similar to that of NDVI except that it uses near-infrared band 5 and the short-wave infrared band 7:

NBR = (b5 - B7) / (B5 + B7)

The NBR equation was designed to be calculated from reflectance, but it can be calculated from radiance and digital_number (dn) with changes to the burn severity (discussed in the table below). For a given area, an NBR is calculated from an image just prior to the burn and a second NBR is calculated for an image immediately following the burn. Burn extent and severity is evaluated by taking the difference between these two index layers:

NBR = NBR(prefire) - NBR(posttfire)

The meaning of the ∆NBR values can vary by scene, and interpretation in specific instances should always be based on some field assessment. However, the following table from the USGS FireMon program can be useful as a first approximation for interpreting the NBR difference:

-2.0 to 0.1 = Regrowth and unburned 0.1 to 0.27 = Low severity burn 0.27 to 0.44 = medium severity burn 0.44 to 0.66 = moderate severity burn '> 0.66 = High severity burn

Source: http://wiki.landscapetoolbox.org/doku.php/remote_sensing_methods:normalized_burn_ratio

In [15]:
hh_prefire  = ((bl8band5raster - bl8band7raster)/(bl8band5raster + bl8band7raster))
hh_prefire

C:\Users\emmel\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [16]:
hh_postfire  = ((al8band5raster - al8band7raster)/(al8band5raster + al8band7raster))
hh_postfire

C:\Users\emmel\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])